### Disability & Impairment Detection

In [0]:
# Import necessary libraries
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

# Configuration
catalog = "dev_structured"  # Update with your catalog name
schema = "analytics"    # Update with your schema name
table_name = "measureresponses_impairment"
endpoint_name = "databricks-gpt-oss-120b"  # or your preferred endpoint

# Processing settings
USE_SAMPLE = True  # Set to False for full dataset
SAMPLE_SIZE = 1000  # Number of records for testing

print(f"Using endpoint: {endpoint_name}")

Using endpoint: databricks-gpt-oss-120b


In [0]:
# Load the data
df = spark.table(f"{catalog}.{schema}.{table_name}")

# Filter for records with text responses
text_responses = df.filter(
    col("koo_responseextended").isNotNull() & 
    (trim(col("koo_responseextended")) != "")
).select(
    "koo_chimeasureresponseid",
    "koo_clientid",
    "koo_contactid",
    "koo_responseextended",
    "koo_appcode",
    "koo_description",
    "createdon"
)

print(f"Total records with text responses: {text_responses.count():,}")

# Apply sampling if configured
if USE_SAMPLE:
    working_df = text_responses.limit(SAMPLE_SIZE)
    print(f"Processing sample: {SAMPLE_SIZE:,} records")
else:
    working_df = text_responses
    print(f"Processing full dataset")

display(working_df.limit(5))


Total records with text responses: 4,925,323
Processing sample: 1,000 records


koo_chimeasureresponseid,koo_clientid,koo_contactid,koo_responseextended,koo_appcode,koo_description,createdon
0E7B0F3A-3149-4EC2-8DEB-A81F5E0A88BB,DED9D147-9C71-F011-B4CD-000D3ACB2909,3FBC9A6A-589E-4F00-929C-20E6B5FAE654,affirmed and praised parenting,int/counselling,Counselling / emotional support,2025-09-02T00:36:51.820Z
0F047726-D936-4413-8EC8-756A6B883093,81113FF1-0366-F011-BEC2-000D3A6A2546,068E9A38-7099-49ED-B70F-ED25F53D2E57,PL,int/support,Links to community support,2025-09-02T00:36:45.732Z
DC1237C0-845B-4501-8FEE-5C5B3DBD2D5A,627019A3-485D-F011-BEC2-000D3ACB2909,4EAC425C-6147-4231-8A73-AE8908C7B8C7,Receptive of advice and plan Will access plunketline and drop in clinic as needed.,cpo/response,Parent(s) / caregiver response to plan of care,2025-09-02T00:38:42.083Z
12DBF0FB-0744-4F6E-B6A4-25DF643F2A90,0535CBC2-D515-F011-998A-000D3A79F7D3,15EB0E7B-F451-49FD-94E7-A7664B0D54A9,Promoted self care,int/counselling,Counselling / emotional support,2025-09-02T00:40:59.183Z
036485A3-A64D-4677-BDF5-26302AF2A283,B84FFCDB-D7CE-EF11-8EE9-000D3AE05FE9,618E976D-E1D4-4E6E-90FF-3650E01AC57A,Development,int/guidance,Anticipatory guidance,2025-09-02T00:24:57.436Z


#### Step 1: Detect Disability/Impairment Presence

In [0]:
# Step 1: Binary classification - Does this mention disability/impairment?
detection_prompt = """You are analyzing healthcare text data from measureresponses_impairment table to identify mentions of disabilities or impairments.

TASK: Determine if the text mentions ANY disability, impairment, or health condition that affects daily functioning.

Consider:
- Physical disabilities (mobility, motor skills)
- Sensory impairments (vision, hearing)
- Developmental conditions (autism, ADHD, learning difficulties)
- Intellectual disabilities
- Mental health conditions (when affecting function)
- Speech/language difficulties
- Chronic health conditions affecting function
- Both current conditions AND past conditions ("used to need...", "no longer requires...")

TEXT: {text}

Respond with ONLY one word:
- "YES" if disability/impairment is mentioned
- "NO" if no disability/impairment is mentioned
"""

disability_detected = working_df.withColumn(
    "disability_mentioned",
    expr(f"""
        ai_query(
            '{endpoint_name}',
            '{detection_prompt.replace("'", "\\'").replace("{text}", "' || koo_responseextended || '")}'
        )
    """)
).withColumn(
    "disability_mentioned",
    upper(trim(col("disability_mentioned")))
)

# Show distribution
print("\n=== Disability Mention Detection ===")
display(
    disability_detected
    .groupBy("disability_mentioned")
    .count()
    .orderBy(col("count").desc())
)

# Filter to only records with disability mentions
disability_records = disability_detected.filter(col("disability_mentioned") == "YES")
print(f"\nRecords mentioning disability: {disability_records.count():,}")


=== Disability Mention Detection ===


disability_mentioned,count
NO,897
YES,103



Records mentioning disability: 96


#### Step 2: Extract and Classify Disability Information

In [0]:
# Step 2: Detailed extraction prompt
extraction_prompt = """You are extracting disability information from measureresponses_impairment table.

TEXT: {text}

Extract the following information in JSON format:
{{
  "disability_type": "<primary category>",
  "specific_condition": "<specific condition mentioned>",
  "duration": "<permanent|transient|past|unknown>",
  "severity": "<mild|moderate|severe|unknown>",
  "support_needs": "<brief description of support mentioned>",
  "who_affected": "<child|parent|family_member|unknown>"
}}

DISABILITY TYPE categories:
- "physical": Mobility, motor skills, physical function
- "sensory_vision": Vision impairments, blindness
- "sensory_hearing": Hearing impairments, deafness
- "developmental": Autism, ADHD, developmental delays
- "intellectual": Cognitive disabilities, learning difficulties
- "speech_language": Communication difficulties
- "mental_health": Mental health conditions affecting function
- "chronic_health": Chronic conditions affecting daily life
- "multiple": Multiple types mentioned

DURATION:
- "permanent": Ongoing, chronic, lifelong
- "transient": Temporary, "going away", "improving"
- "past": "used to", "no longer", "previously"
- "unknown": Cannot determine

Respond with ONLY the JSON object, no additional text.
"""

# Apply extraction (sample first 1000 records for testing, remove .limit() for full dataset)
disability_classified = disability_records.limit(1000).withColumn(
    "disability_details",
    expr(f"""
        ai_query(
            '{endpoint_name}',
            '{extraction_prompt.replace("'", "\\'").replace("{text}", "' || koo_responseextended || '")}'
        )
    """)
)

# Parse JSON response
disability_parsed = disability_classified.withColumn(
    "disability_type", 
    expr("get_json_object(disability_details, '$.disability_type')")
).withColumn(
    "specific_condition",
    expr("get_json_object(disability_details, '$.specific_condition')")
).withColumn(
    "duration",
    expr("get_json_object(disability_details, '$.duration')")
).withColumn(
    "severity",
    expr("get_json_object(disability_details, '$.severity')")
).withColumn(
    "who_affected",
    expr("get_json_object(disability_details, '$.who_affected')")
)

print("\n=== Sample Extracted Disability Information ===")
display(
    disability_parsed.select(
        "koo_responseextended",
        "disability_type",
        "specific_condition",
        "duration",
        "who_affected"
    ).limit(10)
)


=== Sample Extracted Disability Information ===


koo_responseextended,disability_type,specific_condition,duration,who_affected
HC and length discrepancy - recheck reflux age-related injuries disrupted sleep and settling behaviour,chronic_health,"reflux, length discrepancy, age-related injuries, disrupted sleep and settling behaviour",unknown,child
"mum mentioned that physio came every two weeks before. Aiden is now able to swallow more effectively. He is eating porridge, mashed meat, tolerating eggs, and able to drink Is applying child disability support, will ask letter from pediatrician or physio",physical,swallowing difficulty (feeding/dysphagia),transient,child
Recent spate of injuries to Eli and Daemon- # leg and neck injury. However mostly well apart from Daemons ears that keep giving him problems,multiple,leg and neck injuries; ear problems,transient,child
Paternal bipolar schizophrenia.,mental_health,bipolar schizophrenia,permanent,parent
foc and 1st child wears glasses,sensory_vision,wears glasses,permanent,child
waking 3-4 hourly overnight 30min-2 hour long day sleeps,chronic_health,sleep disturbance,unknown,child
R hearing loss.,sensory_hearing,right hearing loss,unknown,unknown
Paternal grandparents wear glasses,sensory_vision,vision impairment (requires glasses),permanent,family_member
tongue tie small steady weight gain,speech_language,tongue tie,unknown,child
"I have a fast flow and hes been choking. hes been really uncomfortable think he has reflux, started infocol and we are both sleeping better. He has a cough and he still sounds congested but no fever. He likes to be held a lot Id like to offer a bottle what do you sugeest Im going back to work earlier this time part time in Feb",chronic_health,gastroesophageal reflux (reflux) with cough and congestion,transient,child


#### Step 3: Analyze Distribution and Patterns

In [0]:
if disability_count > 0:
    # Filter to successfully parsed records
    valid_parsed = disability_parsed.filter(col("disability_type").isNotNull())
    
    # 1. Distribution by disability type
    print("=== Distribution by Disability Type ===")
    type_dist = valid_parsed.groupBy("disability_type").count().orderBy(col("count").desc())
    display(type_dist)
    
    # 2. Distribution by duration (permanent vs transient)
    print("\n=== Distribution by Duration ===")
    duration_dist = valid_parsed.groupBy("duration").count().orderBy(col("count").desc())
    display(duration_dist)
    
    # 3. Cross-tab: Type vs Duration
    print("\n=== Disability Type vs Duration ===")
    crosstab = valid_parsed.groupBy("disability_type", "duration").count().orderBy("disability_type", col("count").desc())
    display(crosstab)
    
    # 4. Who is affected
    print("\n=== Who is Affected ===")
    affected_dist = valid_parsed.groupBy("who_affected").count().orderBy(col("count").desc())
    display(affected_dist)
    
    # 5. Severity distribution
    print("\n=== Severity Distribution ===")
    severity_dist = valid_parsed.groupBy("severity").count().orderBy(col("count").desc())
    display(severity_dist)
else:
    print("No data to analyze.")

In [0]:
# Distribution by disability type
print("\n=== Distribution by Disability Type ===")
display(
    disability_parsed
    .groupBy("disability_type")
    .count()
    .orderBy(col("count").desc())
)

# Distribution by duration (permanent vs transient)
print("\n=== Distribution by Duration ===")
display(
    disability_parsed
    .groupBy("duration")
    .count()
    .orderBy(col("count").desc())
)

# Cross-tabulation: Type vs Duration
print("\n=== Disability Type vs Duration ===")
display(
    disability_parsed
    .groupBy("disability_type", "duration")
    .count()
    .orderBy("disability_type", col("count").desc())
)

# Who is affected
print("\n=== Who is Affected ===")
display(
    disability_parsed
    .groupBy("who_affected")
    .count()
    .orderBy(col("count").desc())
)


=== Distribution by Disability Type ===


disability_type,count
chronic_health,26
physical,22
speech_language,15
sensory_vision,13
mental_health,9
multiple,8
developmental,3
intellectual,3
sensory_hearing,1



=== Distribution by Duration ===


duration,count
unknown,59
permanent,27
transient,6
past,3



=== Disability Type vs Duration ===


disability_type,duration,count
chronic_health,unknown,13
chronic_health,permanent,5
chronic_health,transient,5
chronic_health,past,3
developmental,unknown,2
developmental,permanent,1
intellectual,permanent,3
mental_health,unknown,7
mental_health,permanent,1
multiple,unknown,3



=== Who is Affected ===


who_affected,count
child,72
family_member,13
parent,11
unknown,9


#### Step 4: Identify Priority Populations - Permanent Disabilities

In [0]:
# Focus on permanent disabilities for priority population identification
permanent_disabilities = disability_parsed.filter(
    col("duration").isin(["permanent", "unknown"])  # Include unknown as potentially permanent
)

print(f"\n=== Priority Population: Permanent/Unknown Duration Disabilities ===")
print(f"Total records: {permanent_disabilities.count():,}")

# Top specific conditions
print("\nTop Specific Conditions:")
display(
    permanent_disabilities
    .groupBy("specific_condition")
    .count()
    .orderBy(col("count").desc())
    .limit(20)
)


=== Priority Population: Permanent/Unknown Duration Disabilities ===
Total records: 86

Top Specific Conditions:


specific_condition,count
mild cataracts,4
xyy syndrome,4
postnatal depression,4
thyroid disorder,3
mental health condition,3
breast cancer (stage 3),3
uneven hip creases,3
CASK genetic mutation,3
"preterm birth, slow weight gain, tongue tie",2
"anxiety, depression, personality disorder",2
